In [74]:
%pip install pinecone-client modal sentence-transformers

  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached transformers-4.35.0-py3-none-any.whl.metadata (123 kB)
  Using cached torch-2.1.0-cp311-none-macosx_11_0_arm64.whl.metadata (24 kB)
  Using cached scikit_learn-1.3.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.11.3-cp311-cp311-macosx_12_0_arm64.whl.metadata (165 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached sentencepiece-0.1.99-cp311-cp311-macosx_11_0_arm64.whl (1.2 MB)
  Using cached huggingface_hub-0.19.0-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2023.10.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached tokenizers-0.14.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached safetensors-0.4.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
  Using cached huggingface_hub-0.17.3-p

In [75]:
import os
import pinecone
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")
if not api_key:
    print("Warning: No Pinecone API key found.")
modal_token_id = os.getenv("MODAL_TOKEN_ID")
if not modal_token_id:
    print("Warning: No MODAL_TOKEN_ID found.")
modal_token_secret = os.getenv("MODAL_TOKEN_SECRET")
if not modal_token_secret:
    print("Warning: No MODAL_TOKEN_SECRET found.")

pinecone.init(api_key=os.getenv(api_key), environment="us-west1-gcp-free")


In [76]:
if "cs191" not in pinecone.list_indexes():
    pinecone.create_index("cs191", dimension=512)

In [82]:
STATIC_DIR = os.path.dirname(os.path.abspath(os.curdir))
FLAKS_DIR = os.path.dirname(STATIC_DIR)
ROOT_DIRECTORY = os.path.dirname(FLAKS_DIR)
INSTANCE_DIR = os.path.join(ROOT_DIRECTORY, "instance")
DEMO_DIR = os.path.join(INSTANCE_DIR, "demo")
MODAL_REMOTE_PREFIX = "/root/converted_photos"

('/Users/kennyosele/Documents/Projects/cs191',
 '/Users/kennyosele/Documents/Projects/cs191/instance')

In [57]:
# Embeddings Generation
from modal.object import Object
from modal import Stub, Image, Mount, method

stub = Stub()


def download_models():
    # Caches the model inside the Modal image, so subsequent cold starts are faster.
    from sentence_transformers import SentenceTransformer

    SentenceTransformer("sentence-transformers/clip-ViT-B-32")


container_image = (
    Image.debian_slim()
    .pip_install("sentence-transformers")
    .run_function(download_models)
)


@stub.cls(
    image=container_image,
    mounts=[
        # JIT Mounting of needed files
        Mount.from_local_dir(
            INSTANCE_DIR, remote_path="/root/instance"
        ),
        Mount.from_local_dir(
            local_path=DEMO_DIR,
            remote_path=MODAL_REMOTE_PREFIX,
        ),
    ],
)
class ModalEmbedding:
    def __enter__(self):
        from sentence_transformers import SentenceTransformer

        self.model = SentenceTransformer("sentence-transformers/clip-ViT-B-32")

    @method()
    def generate(self, names_of_files_to_embed):
        from tqdm import tqdm
        import json

        # Generate embeddings for each image
        # with open(f'/root/instance/embeddings.json', 'r') as json_file:
        embeddings = {}
        for image_name in tqdm(names_of_files_to_embed, desc="Generating embeddings"):
            if image_name in embeddings:
                continue
            import PIL.Image

            image_path = f"{MODAL_REMOTE_PREFIX}/{image_name}"
            try:
                image = PIL.Image.open(image_path).convert("RGB")
            except Exception as exc:
                print(image_path, exc)
                return None


            img_emb = self.model.encode(image).tolist()
            embeddings[image_name] = {"clip_embeddings": img_emb}

        return embeddings


In [9]:
def generate_embedding(query):
    from sentence_transformers import SentenceTransformer, util
    import numpy

    model = SentenceTransformer("clip-ViT-B-32")
    # First, we encode the query (which can either be an image or a text string)
    query_emb = model.encode([query], show_progress_bar=False)
    return numpy.ndarray.tolist(query_emb)